In [77]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip

In [6]:
# load the 2017 data
scripts_file = gzip.open('./dw-data/201701scripts_sample.csv.gz', 'r')
    
scripts = pd.read_csv(scripts_file)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [7]:
# col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
# practices = pd.read_csv("dw-data/practices.csv.gz")
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [8]:
CHEM = pd.read_csv("dw-data/chem.csv.gz")
CHEM.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [9]:
def describe(key):
    desc = scripts[key].describe()
    total = desc['count'] * desc['mean']

    return (total, desc['mean'], desc['std'], desc['25%'], desc['50%'], desc['75%'])



summary_stats = [('items', describe('items')),('quantity', describe('quantity')), ('nic', describe('nic')), ('act_cost', describe('act_cost'))]

In [10]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [11]:
grpd_df = scripts.groupby('bnf_name')['items'].sum()

max_n = grpd_df.max()

most_common_item = [(grpd_df.index[ grpd_df == max_n][0], max_n)]

In [12]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [13]:
group1=practices.groupby('code')['post_code'].min().reset_index()

group2=scripts.merge(group1, left_on='practice', right_on='code')

sum_bnf = group2.groupby(['post_code','bnf_name'])['items'].sum().reset_index()

sum_bnf_name = sum_bnf.sort_values('items', ascending = False).groupby('post_code').first()

sum_overall = group2.groupby('post_code')['items'].sum().reset_index()

merged_q3 = sum_bnf_name.merge(sum_overall, on = 'post_code')

merged_q3['proportion'] = merged_q3['items_x'] / merged_q3['items_y']

merged_Q3 = merged_q3[['post_code', 'bnf_name', 'proportion']].head(100)


In [14]:

items_by_region = [row[1:] for row in merged_Q3.itertuples()]

In [15]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

In [16]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [67]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [68]:
chem = pd.read_csv(gzip.open('./dw-data/chem.csv.gz', 'r'))
chem.head()


,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [69]:
ops = '|'.join(opioids)

bnf_by_practice = scripts.groupby('practice').agg({'bnf_code': 'count'}).rename(columns={'bnf_code':'bnf_sum'})


scripts_chem = pd.merge(scripts, chem, left_on='bnf_code', right_on='CHEM SUB', how='left', sort=False)
scripts_chem['opioid?'] = (scripts_chem['NAME'].str.contains(ops, case=False) | scripts_chem['bnf_name'].str.contains(ops, case=False))

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [70]:
opioids_per_practice = scripts_chem.groupby('practice')['opioid?'].mean()
overall_opioids_rate = scripts_chem['opioid?'].mean()

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [71]:
relative_opioids_per_practice = opioids_per_practice - overall_opioids_rate

overall_opioids_std = scripts_chem['opioid?'].std()

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [72]:
standard_error_per_practice = overall_opioids_std / np.sqrt(bnf_by_practice.bnf_sum)

opioid_scores = (relative_opioids_per_practice / standard_error_per_practice).reset_index(name='opioid_scores')


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [73]:

# unique_practices = ...
# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100
unique_practices = practices.sort_values(by='name').groupby('code', sort=False).first()
unique_practices.reset_index(inplace=True)

final_merge = opioid_scores.merge(unique_practices[['code','name']], left_on='practice',
                                  right_on='code', how='left')

results = final_merge.merge(bnf_by_practice.reset_index(), on='practice')

subset = results[['name', 'opioid_scores', 'bnf_sum']]
anomalies = [tuple(val) for val in subset.values]
anomalies = sorted(anomalies, key=lambda x: x[1], reverse=True)[:100]

In [74]:
results.head()

,practice,opioid_scores,code,name,bnf_sum
0,A81005,-0.553511,A81005,SPRINGWOOD SURGERY,1507
1,A81007,1.536807,A81007,BANKHOUSE SURGERY,1454
2,A81011,2.290292,A81011,CHADWICK PRACTICE,1568
3,A81012,1.371001,A81012,WESTBOURNE MEDICAL CENTRE,1332
4,A81017,0.583831,A81017,WOODBRIDGE PRACTICE,2150


In [75]:
grader.score.dw__script_anomalies(anomalies)

Your score:  0.9900000000000007


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [39]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')

In [40]:
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [41]:
# calculate the growth rate
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()

In [42]:
# Combine the two series in a Dataframe
resp = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()

In [43]:
#Get rid of Nans
resp.fillna(0, inplace=True)

In [44]:
#Remove elements with less then 50 prescriptions
resp = resp[resp['item16'] >= 50]

In [45]:

#Sort by growth rate
resp.sort_values(by='growth', ascending = False, inplace=True)

In [46]:
#Pick the top 50 and last 50 and get the tuple list
script_growth = list(pd.concat([resp.head(50), resp.tail(50)]).itertuples(index=False, name=None))

In [47]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [49]:
practices_unique = practices.sort_values(['code', 'post_code']).drop_duplicates(subset='code')

group_by_bnf_code = scripts.groupby(['bnf_code'])['bnf_code'].count().reset_index(name='bnf_code_count')
group_by_bnf_code['p'] = group_by_bnf_code['bnf_code_count'] / len(scripts)
px = 1 / len(group_by_bnf_code)

group_by_bnf_code['rare'] = np.where(group_by_bnf_code['p'] < 0.1 * px, True, False)
scripts_rare = pd.merge(scripts, group_by_bnf_code, how = 'inner', on = 'bnf_code')

In [50]:
total_cost_by_practice = scripts_rare.groupby('practice')['act_cost'].sum().reset_index(name = 'total_cost')

rare_cost_by_practice = scripts_rare[scripts_rare['rare'] == True].groupby(
    'practice')['act_cost'].sum().reset_index(name = 'rare_cost')

script_cost_by_practice = pd.merge(total_cost_by_practice, rare_cost_by_practice, on = 'practice', how='left')

script_cost_by_practice.fillna(0, inplace=True)

script_cost_by_practice['rare_cost_prop'] = (
    script_cost_by_practice['rare_cost'] / script_cost_by_practice['total_cost'])

In [51]:
total_cost = scripts_rare['act_cost'].sum()
total_rare_cost = scripts_rare[scripts_rare['rare'] == True]['act_cost'].sum()

overall_rare_cost = total_rare_cost / total_cost
script_cost_by_practice['relative_rare_cost_prop'] = script_cost_by_practice['rare_cost_prop'] - overall_rare_cost

In [52]:
script_cost_by_practice['standard_errors'] = script_cost_by_practice['relative_rare_cost_prop'].std()

In [53]:
script_cost_by_practice['z_scores'] = script_cost_by_practice['relative_rare_cost_prop'] / script_cost_by_practice['standard_errors']
print(script_cost_by_practice.shape)
practices_unique = practices.sort_values('name').groupby('code', sort=False).first()
practices_unique.reset_index(inplace=True)
merge_script_practice_z_scores = script_cost_by_practice.merge(
    practices_unique[['name', 'code']], how = 'inner', left_on = 'practice', right_on = 'code', sort = False)

print(merge_script_practice_z_scores.shape)
merge_script_practice_z_scores = merge_script_practice_z_scores.sort_values('name', ascending = True)
merge_script_practice_z_scores.drop_duplicates(subset = ['code', 'name'], inplace = True)

(856, 7)
(856, 9)


In [63]:
rare_scripts = []
for index, row in merge_script_practice_z_scores.iterrows():
    rare_scripts.append(
        (row['practice'], 
         row['name'],
         row['z_scores']))

rare_scripts = sorted(rare_scripts, key=lambda x: x[2], reverse = True)[:100]

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [64]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [65]:
grader.score('dw__rare_scripts', rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*